# Cart-pole (collocation)

In this notebook, we solve a cart-pole swing-up problem using direct collocation: an
alternative to single shooting that optimizes the full state trajectory simultaneously.

Features used:
- {class}`~jaxls.Var` subclassing with batched IDs for state and control trajectories
- {func}`@jaxls.Cost.factory <jaxls.Cost.factory>` with batched arguments
- Equality constraints (`constraint_eq_zero`): dynamics collocation
- Inequality constraints (`constraint_leq_zero`): control bounds
- Augmented Lagrangian solver for constrained optimization

In [ ]:
import sys
from loguru import logger

logger.remove()
logger.add(sys.stdout, format="<level>{level: <8}</level> | {message}");

In [2]:
import jax
import jax.numpy as jnp
import jaxls

## Cart-pole dynamics

The cart-pole system has state $[x, \theta, \dot{x}, \dot{\theta}]$ where $x$ is cart position and $\theta$ is pole angle (0 = hanging down, $\pi$ = upright). The control input is a horizontal force on the cart.

In [3]:
# Physical parameters.
m_cart = 1.0  # Cart mass (kg)
m_pole = 0.1  # Pole mass (kg)
length = 0.5  # Pole half-length (m)
gravity = 9.81  # Gravitational acceleration (m/s^2)
force_limit = 10.0  # Maximum control force (N)

# Trajectory parameters.
n_swing = 50  # Timesteps for swing-up phase
n_hold = 10  # Timesteps to hold upright position
n_steps = n_swing + n_hold  # Total timesteps
dt = 0.05  # Time step (s)
total_time = n_steps * dt
print(f"Total trajectory time: {total_time:.2f}s ({n_swing} swing + {n_hold} hold)")

Total trajectory time: 3.00s (50 swing + 10 hold)


In [4]:
@jax.jit
def cart_pole_dynamics(state: jax.Array, force: jax.Array) -> jax.Array:
    """Compute state derivatives for the cart-pole system.

    State: [x, theta, x_dot, theta_dot]
    theta = 0: pole hanging down, theta = pi: pole upright

    Note: We use the standard cart-pole equations but with theta measured
    from hanging down, so the gravitational term has a negative sign.
    """
    x, theta, x_dot, theta_dot = state
    f = force[0]

    sin_th = jnp.sin(theta)
    cos_th = jnp.cos(theta)

    # Total mass.
    total_mass = m_cart + m_pole
    pole_mass_length = m_pole * length

    # Equations of motion for theta=0 being DOWN (stable equilibrium)
    # The gravitational term is negated compared to theta=0 being UP.
    temp = (f + pole_mass_length * theta_dot**2 * sin_th) / total_mass
    theta_ddot = (-gravity * sin_th - cos_th * temp) / (
        length * (4.0 / 3.0 - m_pole * cos_th**2 / total_mass)
    )
    x_ddot = temp - pole_mass_length * theta_ddot * cos_th / total_mass

    return jnp.array([x_dot, theta_dot, x_ddot, theta_ddot])


@jax.jit
def rk4_step(state: jax.Array, force: jax.Array, dt: float) -> jax.Array:
    """Runge-Kutta 4th order integration step."""
    k1 = cart_pole_dynamics(state, force)
    k2 = cart_pole_dynamics(state + 0.5 * dt * k1, force)
    k3 = cart_pole_dynamics(state + 0.5 * dt * k2, force)
    k4 = cart_pole_dynamics(state + dt * k3, force)
    return state + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)

## Variables

We define state variables at each timestep and control inputs between timesteps. Using batched IDs for efficient construction:

In [5]:
class StateVar(jaxls.Var[jax.Array], default_factory=lambda: jnp.zeros(4)):
    """State variable: [x, theta, x_dot, theta_dot]."""


class ControlVar(jaxls.Var[jax.Array], default_factory=lambda: jnp.zeros(1)):
    """Control variable: [force]."""


# Create batched variables.
state_vars = StateVar(id=jnp.arange(n_steps + 1))  # States at t=0, 1, ..., n_steps
control_vars = ControlVar(id=jnp.arange(n_steps))  # Controls at t=0, 1, ..., n_steps-1

print(f"State variables: {n_steps + 1} (shape: {state_vars.id.shape})")
print(f"Control variables: {n_steps} (shape: {control_vars.id.shape})")

State variables: 61 (shape: (61,))
Control variables: 60 (shape: (60,))


## Cost functions

We define costs for:
1. Dynamics constraints: RK4 integration between consecutive states
2. Boundary costs: Penalize deviation from initial and target states
3. Control effort: Minimize force usage
4. Control bounds: Keep force within limits

In [6]:
@jaxls.Cost.factory(kind="constraint_eq_zero")
def dynamics_constraint(
    vals: jaxls.VarValues,
    state_k: StateVar,
    state_k1: StateVar,
    control_k: ControlVar,
    dt: float,
) -> jax.Array:
    """Enforce RK4 dynamics between consecutive states."""
    s_k = vals[state_k]
    s_k1 = vals[state_k1]
    u_k = vals[control_k]
    s_next = rk4_step(s_k, u_k, dt)
    return s_k1 - s_next


@jaxls.Cost.factory
def boundary_cost(
    vals: jaxls.VarValues,
    var: StateVar,
    target: jax.Array,
) -> jax.Array:
    """Penalize deviation from target state."""
    return (vals[var] - target) * 100.0


@jaxls.Cost.factory
def control_cost(
    vals: jaxls.VarValues,
    var: ControlVar,
) -> jax.Array:
    """Minimize control effort."""
    return vals[var] * 1.0


@jaxls.Cost.factory(kind="constraint_leq_zero")
def control_upper_bound(
    vals: jaxls.VarValues,
    var: ControlVar,
    limit: float,
) -> jax.Array:
    """Control <= limit."""
    return vals[var] - limit


@jaxls.Cost.factory(kind="constraint_leq_zero")
def control_lower_bound(
    vals: jaxls.VarValues,
    var: ControlVar,
    limit: float,
) -> jax.Array:
    """-limit <= control, i.e., -limit - control <= 0."""
    return -limit - vals[var]

## Problem construction

Initial state: pole hanging down ($\theta = 0$). Target state: pole upright ($\theta = \pi$), centered at origin with zero velocity.

In [7]:
# Boundary conditions.
initial_state = jnp.array([0.0, 0.0, 0.0, 0.0])  # x=0, theta=0 (down), zero velocity
target_state = jnp.array([0.0, jnp.pi, 0.0, 0.0])  # x=0, theta=pi (up), zero velocity

# State indices for dynamics constraints.
state_k_ids = jnp.arange(n_steps)  # 0, 1, ..., n_steps-1
state_k1_ids = jnp.arange(1, n_steps + 1)  # 1, 2, ..., n_steps
control_k_ids = jnp.arange(n_steps)  # 0, 1, ..., n_steps-1

# Hold phase: penalize states during hold to stay at target.
hold_state_ids = jnp.arange(n_swing, n_steps + 1)  # States during hold phase
n_hold_states = len(hold_state_ids)
hold_targets = jnp.tile(target_state, (n_hold_states, 1))  # Broadcast for batching

# Build costs using batched construction.
costs: list[jaxls.Cost] = [
    # Dynamics constraints (batched)
    dynamics_constraint(
        StateVar(id=state_k_ids),
        StateVar(id=state_k1_ids),
        ControlVar(id=control_k_ids),
        dt,
    ),
    # Boundary costs.
    boundary_cost(StateVar(id=0), initial_state),
    # Hold phase: penalize all hold states deviating from target.
    boundary_cost(StateVar(id=hold_state_ids), hold_targets),
    # Control effort (batched)
    control_cost(control_vars),
    # Control bounds (batched)
    control_upper_bound(control_vars, force_limit),
    control_lower_bound(control_vars, force_limit),
]

print(f"Created {len(costs)} batched cost objects")

Created 6 batched cost objects


In [ ]:
# Initial guess: linear interpolation for states, zero controls.
t_interp = jnp.linspace(0, 1, n_steps + 1)[:, None]
initial_states = initial_state + t_interp * (target_state - initial_state)
initial_controls = jnp.zeros((n_steps, 1))

initial_vals = jaxls.VarValues.make(
    [
        state_vars.with_value(initial_states),
        control_vars.with_value(initial_controls),
    ]
)

# Create the problem.
problem = jaxls.LeastSquaresProblem(costs, [state_vars, control_vars])

# Visualize the problem structure structure.
problem.show()

In [ ]:
# Analyze the problem.
problem = problem.analyze()

## Solving

In [9]:
solution = problem.solve(initial_vals)

INFO     | Augmented Lagrangian: initial snorm=7.3573e-01, csupn=7.3573e-01, max_rho=1.0572e+05, constraint_dim=360
INFO     |  step #0: cost=10554.9854 lambd=0.0005 inexact_tol=1.0e-02
INFO     |      - augmented_control_lower_bound(60): 0.00000 (avg 0.00000)
INFO     |      - augmented_dynamics_constraint(60): 1805924.12500 (avg 7524.68408)
INFO     |      - augmented_control_upper_bound(60): 0.00000 (avg 0.00000)
INFO     |      - control_cost(60): 0.00000 (avg 0.00000)
INFO     |      - boundary_cost(12): 10554.98535 (avg 219.89554)
INFO     |  step #1: cost=10554.9854 lambd=0.0010 inexact_tol=1.0e-02
INFO     |      - augmented_control_lower_bound(60): 0.00000 (avg 0.00000)
INFO     |      - augmented_dynamics_constraint(60): 1805924.12500 (avg 7524.68408)
INFO     |      - augmented_control_upper_bound(60): 0.00000 (avg 0.00000)
INFO     |      - control_cost(60): 0.00000 (avg 0.00000)
INFO     |      - boundary_cost(12): 10554.98535 (avg 219.89554)
INFO     |  step #2: cost=1055

## Visualization

Extract solution trajectories and create animated visualization:

In [10]:
# Extract solution trajectories.
states = solution[state_vars]  # Shape: (n_steps+1, 4)
controls = solution[control_vars]  # Shape: (n_steps, 1)

times = jnp.linspace(0, total_time, n_steps + 1)
control_times = jnp.linspace(0, total_time - dt, n_steps)

print(f"Final state: x={float(states[-1, 0]):.3f}, theta={float(states[-1, 1]):.3f}")
print(f"Target theta (pi): {jnp.pi:.3f}")

Final state: x=0.000, theta=3.141
Target theta (pi): 3.142


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML

# Animation of cart-pole swing-up.
cart_y_offset = 1.2  # Raise cart so pole can swing freely above ground.
wheel_radius = 0.05


def create_cart_pole_frame(x: float, theta: float) -> dict:
    """Create a single frame of cart-pole visualization."""
    cart_width, cart_height = 0.4, 0.15
    pole_len = 2 * length  # Full pole length.
    pole_x = x + pole_len * jnp.sin(theta)
    pole_y = cart_y_offset - pole_len * jnp.cos(theta)

    # Wheel positions.
    wheel_y = cart_y_offset - cart_height / 2 - wheel_radius

    return dict(
        cart_x=[
            x - cart_width / 2,
            x + cart_width / 2,
            x + cart_width / 2,
            x - cart_width / 2,
            x - cart_width / 2,
        ],
        cart_y=[
            cart_y_offset - cart_height / 2,
            cart_y_offset - cart_height / 2,
            cart_y_offset + cart_height / 2,
            cart_y_offset + cart_height / 2,
            cart_y_offset - cart_height / 2,
        ],
        pole_x=[x, float(pole_x)],
        pole_y=[cart_y_offset, float(pole_y)],
        mass_x=[float(pole_x)],
        mass_y=[float(pole_y)],
        wheel1_x=[x - cart_width / 3],
        wheel1_y=[wheel_y],
        wheel2_x=[x + cart_width / 3],
        wheel2_y=[wheel_y],
    )


# Create animation frames (use every 2nd timestep for performance).
frame_step = 2
frame_indices = list(range(0, n_steps + 1, frame_step))
frames = []
for i in frame_indices:
    frame_data = create_cart_pole_frame(float(states[i, 0]), float(states[i, 1]))
    frames.append(
        go.Frame(
            data=[
                go.Scatter(
                    x=frame_data["cart_x"],
                    y=frame_data["cart_y"],
                    fill="toself",
                    fillcolor="steelblue",
                    line=dict(color="darkblue", width=2),
                ),
                go.Scatter(
                    x=frame_data["pole_x"],
                    y=frame_data["pole_y"],
                    mode="lines",
                    line=dict(color="sienna", width=8),
                ),
                go.Scatter(
                    x=frame_data["mass_x"],
                    y=frame_data["mass_y"],
                    mode="markers",
                    marker=dict(
                        size=18, color="sienna", line=dict(color="black", width=1)
                    ),
                ),
                go.Scatter(
                    x=frame_data["wheel1_x"],
                    y=frame_data["wheel1_y"],
                    mode="markers",
                    marker=dict(size=12, color="black", symbol="circle"),
                ),
                go.Scatter(
                    x=frame_data["wheel2_x"],
                    y=frame_data["wheel2_y"],
                    mode="markers",
                    marker=dict(size=12, color="black", symbol="circle"),
                ),
            ],
            name=str(i),
        )
    )

# Initial frame.
init_frame = create_cart_pole_frame(float(states[0, 0]), float(states[0, 1]))
rail_y = cart_y_offset - 0.15 / 2 - wheel_radius * 2

fig_anim = go.Figure(
    data=[
        go.Scatter(
            x=init_frame["cart_x"],
            y=init_frame["cart_y"],
            fill="toself",
            fillcolor="steelblue",
            line=dict(color="darkblue", width=2),
            name="Cart",
        ),
        go.Scatter(
            x=init_frame["pole_x"],
            y=init_frame["pole_y"],
            mode="lines",
            line=dict(color="sienna", width=8),
            name="Pole",
        ),
        go.Scatter(
            x=init_frame["mass_x"],
            y=init_frame["mass_y"],
            mode="markers",
            marker=dict(size=18, color="sienna", line=dict(color="black", width=1)),
            showlegend=False,
        ),
        go.Scatter(
            x=init_frame["wheel1_x"],
            y=init_frame["wheel1_y"],
            mode="markers",
            marker=dict(size=12, color="black", symbol="circle"),
            showlegend=False,
        ),
        go.Scatter(
            x=init_frame["wheel2_x"],
            y=init_frame["wheel2_y"],
            mode="markers",
            marker=dict(size=12, color="black", symbol="circle"),
            showlegend=False,
        ),
        go.Scatter(
            x=[-2.5, 2.5],
            y=[rail_y, rail_y],
            mode="lines",
            line=dict(color="gray", width=6),
            name="Rail",
        ),
    ],
    frames=frames,
    layout=go.Layout(
        title="Cart-Pole (Collocation)",
        xaxis=dict(
            range=[-2.5, 2.5], title="x (m)", constrain="domain", showgrid=False
        ),
        yaxis=dict(
            range=[-0.1, 2.5],
            title="y (m)",
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
        ),
        updatemenus=[
            dict(
                type="buttons",
                showactive=False,
                y=1.15,
                x=0.5,
                xanchor="center",
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            dict(
                                frame=dict(duration=50, redraw=True),
                                fromcurrent=True,
                                transition=dict(duration=0),
                            ),
                        ],
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[
                            [None],
                            dict(
                                frame=dict(duration=0, redraw=False),
                                mode="immediate",
                            ),
                        ],
                    ),
                ],
            )
        ],
        sliders=[
            dict(
                active=0,
                yanchor="top",
                xanchor="left",
                currentvalue=dict(
                    prefix="Time: ",
                    suffix="s",
                    visible=True,
                    xanchor="center",
                    offset=20,
                ),
                pad=dict(b=10, t=60),
                steps=[
                    dict(
                        args=[
                            [str(i)],
                            dict(
                                frame=dict(duration=0, redraw=True),
                                mode="immediate",
                                transition=dict(duration=0),
                            ),
                        ],
                        label=f"{float(times[i]):.2f}",
                        method="animate",
                    )
                    for i in frame_indices
                ],
                x=0.1,
                y=0,
                len=0.8,
            )
        ],
        height=500,
        showlegend=False,
        margin=dict(t=80, b=100),
        plot_bgcolor="white",
    ),
)
HTML(fig_anim.to_html(full_html=False, include_plotlyjs="cdn", auto_play=False))

In [12]:
# State and control trajectories.
fig_traj = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Cart Position", "Pole Angle", "Velocities", "Control Force"),
    vertical_spacing=0.15,
    horizontal_spacing=0.1,
)

# Cart position.
fig_traj.add_trace(
    go.Scatter(
        x=[float(t) for t in times],
        y=[float(s) for s in states[:, 0]],
        mode="lines",
        line=dict(color="steelblue", width=2),
        name="x",
    ),
    row=1,
    col=1,
)

# Pole angle.
fig_traj.add_trace(
    go.Scatter(
        x=[float(t) for t in times],
        y=[float(s) for s in states[:, 1]],
        mode="lines",
        line=dict(color="coral", width=2),
        name="θ",
    ),
    row=1,
    col=2,
)
fig_traj.add_hline(
    y=float(jnp.pi),
    line_dash="dash",
    line_color="gray",
    row=1,
    col=2,
    annotation_text="π",
)

# Velocities.
fig_traj.add_trace(
    go.Scatter(
        x=[float(t) for t in times],
        y=[float(s) for s in states[:, 2]],
        mode="lines",
        line=dict(color="steelblue", width=2),
        name="ẋ",
    ),
    row=2,
    col=1,
)
fig_traj.add_trace(
    go.Scatter(
        x=[float(t) for t in times],
        y=[float(s) for s in states[:, 3]],
        mode="lines",
        line=dict(color="coral", width=2, dash="dash"),
        name="θ̇",
    ),
    row=2,
    col=1,
)

# Control force.
fig_traj.add_trace(
    go.Scatter(
        x=[float(t) for t in control_times],
        y=[float(u) for u in controls[:, 0]],
        mode="lines",
        line=dict(color="forestgreen", width=2),
        name="F",
        fill="tozeroy",
        fillcolor="rgba(34, 139, 34, 0.2)",
    ),
    row=2,
    col=2,
)
fig_traj.add_hline(
    y=force_limit, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2
)
fig_traj.add_hline(
    y=-force_limit, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2
)

# Axis labels.
fig_traj.update_xaxes(title_text="Time (s)", row=1, col=1)
fig_traj.update_yaxes(title_text="Position (m)", row=1, col=1)
fig_traj.update_xaxes(title_text="Time (s)", row=1, col=2)
fig_traj.update_yaxes(title_text="Angle (rad)", row=1, col=2)
fig_traj.update_xaxes(title_text="Time (s)", row=2, col=1)
fig_traj.update_yaxes(title_text="Velocity", row=2, col=1)
fig_traj.update_xaxes(title_text="Time (s)", row=2, col=2)
fig_traj.update_yaxes(title_text="Force (N)", row=2, col=2)

fig_traj.update_layout(height=500, showlegend=False, margin=dict(t=40, b=40))
HTML(fig_traj.to_html(full_html=False, include_plotlyjs="cdn"))